In [1]:
import time
import random
import pandas as pd
import numpy as np

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, MaxPooling2D, Conv2D, Dropout, Lambda, Dense, Flatten, Activation, Input, Embedding, BatchNormalization
from tensorflow.keras.initializers import glorot_normal, Zeros, TruncatedNormal
from tensorflow.keras.regularizers import l2


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy


from tensorflow.keras.optimizers import Adam
from collections import defaultdict
import math

In [2]:
data_path = './autoint/ml-1m'

In [3]:
# 1. 데이터 불러오기
# csv 데이터이므로 read_csv로 가져옵니다.
movielens_rcmm = pd.read_csv(f"{data_path}/movielens_rcmm_v2.csv", dtype=str)
print(movielens_rcmm.shape)
movielens_rcmm.head()

(1000209, 15)


,user_id,movie_id,movie_decade,movie_year,rating_year,rating_month,rating_decade,genre1,genre2,genre3,gender,age,occupation,zip,label
0,1,1193,1970s,1975,2000,12,2000s,Drama,no,no,F,1,10,48067,1
1,1,661,1990s,1996,2000,12,2000s,Animation,Children's,Musical,F,1,10,48067,0
2,1,914,1960s,1964,2000,12,2000s,Musical,Romance,no,F,1,10,48067,0
3,1,3408,2000s,2000,2000,12,2000s,Drama,no,no,F,1,10,48067,1
4,1,2355,1990s,1998,2001,1,2000s,Animation,Children's,Comedy,F,1,10,48067,1


In [4]:
# 2. 라벨 인코더(label encoder)
# sklearn의 LabelEncoder(https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)
# label은 제외한 각 컬럼을 돌면서 각각의 고윳값들을 0부터 n까지 매핑시킵니다.
label_encoders = {col: LabelEncoder() for col in movielens_rcmm.columns[:-1]} # label은 제외

for col, le in label_encoders.items():
    movielens_rcmm[col] = le.fit_transform(movielens_rcmm[col])

In [5]:
movielens_rcmm['label'] = movielens_rcmm['label'].astype(np.float32)

In [6]:
# 3. 학습 데이터와 테스트데이터로 분리, 0.2 정도로 분리
train_df, test_df = train_test_split(movielens_rcmm, test_size=0.2, random_state=42)

In [7]:
# 필요 컬럼들과 레이블 정의
# 필드의 각 고유 개수를 정의하는 field_dims를 정의합니다. 이는  임베딩 때 활용됩니다. 
u_i_feature = ['user_id', 'movie_id']
meta_features = ['movie_decade', 'movie_year', 'rating_year', 'rating_month', 'rating_decade', 'genre1','genre2', 'genre3', 'gender', 'age', 'occupation', 'zip']
label = 'label'
field_dims = np.max(movielens_rcmm[u_i_feature + meta_features].astype(np.int64).values, axis=0) + 1
field_dims

array([6040, 3706,   10,   81,    4,   12,    1,   18,   18,   16,    2,
          7,   21, 3439])

In [8]:
# 에포크, 학습률, 드롭아웃, 배치사이즈, 임베딩 크기 등 정의
epochs=5
learning_rate= 0.0001
dropout= 0.4
batch_size = 2048
embed_dim= 16

In [9]:
class AutoIntMLP(Layer): 
    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=2, att_res=True, dnn_hidden_units=(32, 32), dnn_activation='relu',
                 l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=False, dnn_dropout=0.4, init_std=0.0001):
        super(AutoIntMLP, self).__init__()
        self.embedding = FeaturesEmbedding(field_dims, embedding_size)
        self.num_fields = len(field_dims)
        self.embedding_size = embedding_size

        self.final_layer = Dense(1, use_bias=False, kernel_initializer=tf.random_normal_initializer(stddev=init_std))
        
        self.dnn = MultiLayerPerceptron(self.embed_output_dim, dnn_hidden_units,
                           activation=dnn_activation, l2_reg=l2_reg_dnn, dropout_rate=dnn_dropout, use_bn=dnn_use_bn,
                           init_std=init_std, output_layer=True, device=device)
        
        self.int_layers = [MultiHeadSelfAttention(att_embedding_size=embedding_size, head_num=att_head_num, use_res=att_res) for _ in range(att_layer_num)]

    def call(self, inputs):
        embed_x = self.embedding(inputs)
        dnn_embed = tf.reshape(embed_x, shape=(-1, self.embedding_size * self.num_fields))

        att_input = embed_x
        for layer in self.int_layers:
            att_input = layer(att_input)

        att_output = Flatten()(att_input)
        att_output = self.final_layer(att_output)
        
        dnn_output = self.dnn(dnn_embed.view(-1, self.embed_output_dim))
        y_pred = torch.sigmoid(att_output + dnn_output)
        
        return y_pred

In [10]:
class AutoIntMLPModel(Model):
    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=2, att_res=True, 
                 l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=False, dnn_dropout=0.4, init_std=0.0001):
        super(AutoIntMLPModel, self).__init__()
        # 임베딩 레이어를 정의합니다. 
        self.embedding = FeaturesEmbedding(field_dims, embedding_size)
        self.num_fields = len(field_dims)
        self.embedding_size = embedding_size
        # 마지막 출력 레이어를 정의합니다.
        self.final_layer = Dense(1, use_bias=False, kernel_initializer=tf.random_normal_initializer(stddev=init_std))
        # 멀티 레이어 퍼셉트론 레이어를 정의합니다.
        self.int_layers = [MultiHeadSelfAttention(att_embedding_size=embedding_size, head_num=att_head_num, use_res=att_res) for _ in range(att_layer_num)]

    def call(self, inputs):
        # input 데이터에 해당되는 embedding 값을 가져옵니다.
        att_input = self.embedding(inputs)
        # 멀티 헤드 셀프 어텐션 레이어에서 상호작용을 수행합니다.
        for layer in self.int_layers:
            att_input = layer(att_input)

        att_output = Flatten()(att_input)
        # 최종 출력입니다. 
        att_output = self.final_layer(att_output)
        # sigmoid로 예측값을 출력합니다.
        y_pred = tf.nn.sigmoid(att_output)

        return y_pred

In [11]:
class FeaturesEmbedding(Layer):  
    '''
    임베딩 레이어입니다. 
    - 만약 피처(feature) 3개가 각각 10개, 20개, 30개의 고유값을 가진다면 feature_dims는 [10, 20, 30] 형태를 띄게 됩니다.
    - 전체 임베딩을 해야 할 개수는 10+20+30 = 60이므로 '60 x 임베딩_차원_크기'의 행렬이 생성되게 됩니다.
    '''
    def __init__(self, field_dims, embed_dim, **kwargs):
        super(FeaturesEmbedding, self).__init__(**kwargs)
        self.total_dim = sum(field_dims)
        self.embed_dim = embed_dim
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.longlong)
        self.embedding = tf.keras.layers.Embedding(input_dim=self.total_dim, output_dim=self.embed_dim)

    def build(self, input_shape):
        # 임베딩을 빌드하고 초기화합니다.
        self.embedding.build(input_shape)
        self.embedding.set_weights([tf.keras.initializers.GlorotUniform()(shape=self.embedding.weights[0].shape)])

    def call(self, x):
        # 들어온 입력의 임베딩을 가져니다.
        x = x + tf.constant(self.offsets)
        return self.embedding(x)

In [12]:
class MultiHeadSelfAttention(Layer):  
    '''
    멀티 헤드 셀프 어텐션 레이어입니다.
    - 위에 작성한 수식과 같이 동작됩니다.
    - 필요에 따라 잔차 연결(residual connection)도 진행합니다.
    '''
    def __init__(self, att_embedding_size=8, head_num=2, use_res=True, scaling=False, seed=1024, **kwargs):
        if head_num <= 0:
            raise ValueError('head_num must be a int > 0')
        self.att_embedding_size = att_embedding_size
        self.head_num = head_num
        self.use_res = use_res
        self.seed = seed
        self.scaling = scaling
        super(MultiHeadSelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        if len(input_shape) != 3:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 3 dimensions" % (len(input_shape)))
        embedding_size = int(input_shape[-1])
        # 쿼리에 해당하는 매트릭스입니다. 
        self.W_Query = self.add_weight(name='query', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                       dtype=tf.float32,
                                       initializer=TruncatedNormal(seed=self.seed))
        # 키에 해당되는 매트릭스입니다.
        self.W_key = self.add_weight(name='key', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                     dtype=tf.float32,
                                     initializer=TruncatedNormal(seed=self.seed + 1))
        # 값(value)에 해당되는 매트릭스입니다.
        self.W_Value = self.add_weight(name='value', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                       dtype=tf.float32,
                                       initializer=TruncatedNormal(seed=self.seed + 2))
        # 필요하다면 잔차 연결도 할 수 있습니다.
        if self.use_res:
            self.W_Res = self.add_weight(name='res', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                         dtype=tf.float32,
                                         initializer=TruncatedNormal(seed=self.seed))

        super(MultiHeadSelfAttention, self).build(input_shape)

    def call(self, inputs, **kwargs):
        if K.ndim(inputs) != 3:
            raise ValueError("Unexpected inputs dimensions %d, expect to be 3 dimensions" % (K.ndim(inputs)))
        
        # 입력이 들어오면 쿼리, 키, 값(value)에 매칭되어 각각의 값을 가지고 옵니다.
        querys = tf.tensordot(inputs, self.W_Query, axes=(-1, 0))  
        keys = tf.tensordot(inputs, self.W_key, axes=(-1, 0))
        values = tf.tensordot(inputs, self.W_Value, axes=(-1, 0))

        # 헤드 개수에 따라 데이터를 분리해줍니다.
        querys = tf.stack(tf.split(querys, self.head_num, axis=2))
        keys = tf.stack(tf.split(keys, self.head_num, axis=2))
        values = tf.stack(tf.split(values, self.head_num, axis=2))
        
        # 쿼리와 키를 먼저 곱해줍니다. 위 이미지의 식 (5)와 같습니다.
        inner_product = tf.matmul(querys, keys, transpose_b=True)
        if self.scaling:
            inner_product /= self.att_embedding_size ** 0.5
        self.normalized_att_scores =  tf.nn.softmax(inner_product)
        
        # 쿼리와 키에서 나온 어텐션 값을 값(value)에 곱해줍니다. 식 (6)과 같습니다.
        result = tf.matmul(self.normalized_att_scores, values)
        # 식 (7)과 같이 쪼개어진 멀테 헤드를 모아줍니다.
        result = tf.concat(tf.split(result, self.head_num, ), axis=-1)
        result = tf.squeeze(result, axis=0) 

        if self.use_res:
            result += tf.tensordot(inputs, self.W_Res, axes=(-1, 0))
        result = tf.nn.relu(result)
        
        # 그 결과 값을 리턴합니다.

        return result

    def compute_output_shape(self, input_shape):

        return (None, input_shape[1], self.att_embedding_size * self.head_num)

    def get_config(self, ):
        config = {'att_embedding_size': self.att_embedding_size, 'head_num': self.head_num, 'use_res': self.use_res,'seed': self.seed}
        base_config = super(MultiHeadSelfAttention, self).get_config()
        base_config.update(config)
        return base_config

In [13]:
autoIntMLP_model = AutoIntMLPModel(field_dims, embed_dim, att_layer_num=3, att_head_num=2, att_res=True,
                             l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=False
                             , dnn_dropout=dropout, init_std=0.0001)

In [14]:
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy(from_logits=False)
# 모델 컴파일
autoIntMLP_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_crossentropy'])

In [15]:
history = autoIntMLP_model.fit(train_df[u_i_feature + meta_features], train_df[label], epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/5
352/352 [==============================] - 7s 14ms/step - loss: 0.6738 - binary_crossentropy: 0.6738 - val_loss: 0.6328 - val_binary_crossentropy: 0.6328
Epoch 2/5
352/352 [==============================] - 4s 12ms/step - loss: 0.6054 - binary_crossentropy: 0.6054 - val_loss: 0.5939 - val_binary_crossentropy: 0.5939
Epoch 3/5
352/352 [==============================] - 4s 12ms/step - loss: 0.5717 - binary_crossentropy: 0.5717 - val_loss: 0.5568 - val_binary_crossentropy: 0.5568
Epoch 4/5
352/352 [==============================] - 4s 12ms/step - loss: 0.5413 - binary_crossentropy: 0.5413 - val_loss: 0.5464 - val_binary_crossentropy: 0.5464
Epoch 5/5
352/352 [==============================] - 4s 12ms/step - loss: 0.5350 - binary_crossentropy: 0.5350 - val_loss: 0.5437 - val_binary_crossentropy: 0.5437


In [16]:
def test_model(model, test_df):
    '''모델 테스트'''
    user_pred_info = defaultdict(list)
    total_rows = len(test_df)
    for i in range(0, total_rows, batch_size):
        features = test_df.iloc[i:i + batch_size, :-1].values
        y_pred = model.predict(features, verbose=False)
        for feature, p in zip(features, y_pred):
            u_i = feature[:2]
            user_pred_info[int(u_i[0])].append((int(u_i[1]), float(p)))
    return user_pred_info

In [17]:
# 사용자에게 예측된 정보를 저장하는 딕셔너리 
user_pred_info = {}
# top10개
top = 10
# 테스트 값을 가지고 옵니다. 
mymodel_user_pred_info = test_model(autoIntMLP_model, test_df)
# 사용자마다 돌면서 예측 데이터 중 가장 높은 top 10만 가져옵니다. 
for user, data_info in tqdm(mymodel_user_pred_info.items(), total=len(mymodel_user_pred_info), position=0, leave=True):
    ranklist = sorted(data_info, key=lambda s : s[1], reverse=True)[:top]
    ranklist = list(dict.fromkeys([r[0] for r in ranklist]))
    user_pred_info[str(user)] = ranklist
# 원본 테스트 데이터에서 label이 1인 사용자 별 영화 정보를 가져옵니다.
test_data = test_df[test_df['label']==1].groupby('user_id')['movie_id'].apply(list)

100%|██████████| 6035/6035 [00:00<00:00, 76173.55it/s]


In [18]:
def get_DCG(ranklist, y_true):
    dcg = 0.0
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item in y_true:
            dcg += 1.0 / math.log(i + 2)
    return  dcg

def get_IDCG(ranklist, y_true):
    idcg = 0.0
    i = 0
    for item in y_true:
        if item in ranklist:
            idcg += 1.0 / math.log(i + 2)
            i += 1
    return idcg

def get_NDCG(ranklist, y_true):
    '''NDCG 평가 지표'''
    ranklist = np.array(ranklist).astype(int)
    y_true = np.array(y_true).astype(int)
    dcg = get_DCG(ranklist, y_true)
    idcg = get_IDCG(y_true, y_true)
    if idcg == 0:
        return 0
    return round( (dcg / idcg), 5)

def get_hit_rate(ranklist, y_true):
    '''hitrate 평가 지표'''
    c = 0
    for y in y_true:
        if y in ranklist:
            c += 1
    return round( c / len(y_true), 5 )

In [19]:
mymodel_ndcg_result = {}
mymodel_hitrate_result = {}

# 모델 예측값과 원본 테스트 데이터를 비교해서 어느정도 성능이 나왔는지 NDCG와 Hitrate를 비교합니다.
# [[YOUR CODE]]

# NDCG 
for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
    mymodel_pred = user_pred_info.get(str(user))

    testset = list(set(np.array(data_info).astype(int)))
    mymodel_pred = mymodel_pred[:top]

    # NDCG 값 구하기
    user_ndcg = get_NDCG(mymodel_pred, testset)

    mymodel_ndcg_result[user] = user_ndcg
    
# Hitrate
for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
    mymodel_pred = user_pred_info.get(str(user))

    testset = list(set(np.array(data_info).astype(int)))
    mymodel_pred = mymodel_pred[:top]

    # hitrate 값 구하기
    user_hitrate = get_hit_rate(mymodel_pred, testset)

    # 사용자 hitrate 결과 저장
    mymodel_hitrate_result[user] = user_hitrate

100%|██████████| 5994/5994 [00:00<00:00, 29121.31it/s]


In [20]:
print(" mymodel ndcg : ", round(np.mean(list(mymodel_ndcg_result.values())), 5))
print(" mymodel hitrate : ", round(np.mean(list(mymodel_hitrate_result.values())), 5))

 mymodel ndcg :  0.66275
 mymodel hitrate :  0.63281


In [21]:
np.save('./autoint/field_dims_mlp.npy', field_dims)

In [22]:
autoIntMLP_model.save_weights('./autoint/autoIntMLP_model_weights.h5')

In [23]:
import joblib 

joblib.dump(label_encoders, './autoint/autoIntMLP_label_encoders.pkl')

['./autoint/autoIntMLP_label_encoders.pkl']

In [30]:
# 에포크, 학습률, 드롭아웃, 배치사이즈, 임베딩 크기 등 정의
epochs=10
learning_rate= 0.0001
dropout= 0.4
batch_size = 2048
embed_dim= 16

In [31]:
autoIntMLP_model = AutoIntMLPModel(field_dims, embed_dim, att_layer_num=3, att_head_num=2, att_res=True,
                             l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=False
                             , dnn_dropout=dropout, init_std=0.0001)

optimizer = Adam(learning_rate=learning_rate)

loss_fn = BinaryCrossentropy(from_logits=False)

# 모델 컴파일
autoIntMLP_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_crossentropy'])
history = autoIntMLP_model.fit(train_df[u_i_feature + meta_features], train_df[label], epochs=epochs, batch_size=batch_size, validation_split=0.1)

# 사용자에게 예측된 정보를 저장하는 딕셔너리 
user_pred_info = {}
# top10개
top = 10
# 테스트 값을 가지고 옵니다. 
mymodel_user_pred_info = test_model(autoIntMLP_model, test_df)

# 사용자마다 돌면서 예측 데이터 중 가장 높은 top 10만 가져옵니다. 
for user, data_info in tqdm(mymodel_user_pred_info.items(), total=len(mymodel_user_pred_info), position=0, leave=True):
    ranklist = sorted(data_info, key=lambda s : s[1], reverse=True)[:top]
    ranklist = list(dict.fromkeys([r[0] for r in ranklist]))
    user_pred_info[str(user)] = ranklist
    
# 원본 테스트 데이터에서 label이 1인 사용자 별 영화 정보를 가져옵니다.
test_data = test_df[test_df['label']==1].groupby('user_id')['movie_id'].apply(list)
mymodel_ndcg_result = {}
mymodel_hitrate_result = {}

# 모델 예측값과 원본 테스트 데이터를 비교해서 어느정도 성능이 나왔는지 NDCG와 Hitrate를 비교합니다.
# [[YOUR CODE]]

# NDCG 
for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
    mymodel_pred = user_pred_info.get(str(user))

    testset = list(set(np.array(data_info).astype(int)))
    mymodel_pred = mymodel_pred[:top]

    # NDCG 값 구하기
    user_ndcg = get_NDCG(mymodel_pred, testset)

    mymodel_ndcg_result[user] = user_ndcg
    
# Hitrate
for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
    mymodel_pred = user_pred_info.get(str(user))

    testset = list(set(np.array(data_info).astype(int)))
    mymodel_pred = mymodel_pred[:top]

    # hitrate 값 구하기
    user_hitrate = get_hit_rate(mymodel_pred, testset)

    # 사용자 hitrate 결과 저장
    mymodel_hitrate_result[user] = user_hitrate
print(" mymodel ndcg : ", round(np.mean(list(mymodel_ndcg_result.values())), 5))
print(" mymodel hitrate : ", round(np.mean(list(mymodel_hitrate_result.values())), 5))    

Epoch 1/10
352/352 [==============================] - 6s 14ms/step - loss: 0.6737 - binary_crossentropy: 0.6737 - val_loss: 0.6314 - val_binary_crossentropy: 0.6314
Epoch 2/10
352/352 [==============================] - 4s 12ms/step - loss: 0.6044 - binary_crossentropy: 0.6044 - val_loss: 0.5940 - val_binary_crossentropy: 0.5940
Epoch 3/10
352/352 [==============================] - 4s 12ms/step - loss: 0.5769 - binary_crossentropy: 0.5769 - val_loss: 0.5643 - val_binary_crossentropy: 0.5643
Epoch 4/10
352/352 [==============================] - 4s 12ms/step - loss: 0.5448 - binary_crossentropy: 0.5448 - val_loss: 0.5476 - val_binary_crossentropy: 0.5476
Epoch 5/10
352/352 [==============================] - 4s 13ms/step - loss: 0.5363 - binary_crossentropy: 0.5363 - val_loss: 0.5448 - val_binary_crossentropy: 0.5448
Epoch 6/10
352/352 [==============================] - 4s 13ms/step - loss: 0.5335 - binary_crossentropy: 0.5335 - val_loss: 0.5431 - val_binary_crossentropy: 0.5431
Epoch 7/10

100%|██████████| 5994/5994 [00:00<00:00, 24790.00it/s]

 mymodel ndcg :  0.6627
 mymodel hitrate :  0.63302


In [32]:
from itertools import product
from tqdm import tqdm
import numpy as np

# 하이퍼파라미터 조합 설정
param_grid = {
    'epochs': [5, 10],
    'learning_rate': [0.0001, 0.001],
    'dropout': [0.3, 0.4],
    'batch_size': [1024, 2048],
    'embed_dim': [16, 32]
}

# 결과 저장 리스트
experiment_results = []

for params in product(*param_grid.values()):
    epochs, learning_rate, dropout, batch_size, embed_dim = params

    print(f"\nRunning with params: epochs={epochs}, lr={learning_rate}, dropout={dropout}, batch_size={batch_size}, embed_dim={embed_dim}")

    # 모델 초기화 및 컴파일
    autoIntMLP_model = AutoIntMLPModel(
        field_dims, embed_dim, att_layer_num=3, att_head_num=2, att_res=True,
        dnn_dropout=dropout, init_std=0.0001
    )
    optimizer = Adam(learning_rate=learning_rate)
    loss_fn = BinaryCrossentropy(from_logits=False)
    autoIntMLP_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_crossentropy'])

    # 모델 훈련
    history = autoIntMLP_model.fit(
        train_df[u_i_feature + meta_features], train_df[label],
        epochs=epochs, batch_size=batch_size, validation_split=0.1
    )

    # 테스트 데이터로 예측 수행
    user_pred_info = {}
    top = 10
    mymodel_user_pred_info = test_model(autoIntMLP_model, test_df)

    for user, data_info in tqdm(mymodel_user_pred_info.items(), total=len(mymodel_user_pred_info), position=0, leave=True):
        ranklist = sorted(data_info, key=lambda s: s[1], reverse=True)[:top]
        ranklist = list(dict.fromkeys([r[0] for r in ranklist]))
        user_pred_info[str(user)] = ranklist

    # 원본 테스트 데이터에서 label이 1인 사용자별 영화 정보를 가져옴
    test_data = test_df[test_df['label'] == 1].groupby('user_id')['movie_id'].apply(list)
    mymodel_ndcg_result = {}
    mymodel_hitrate_result = {}

    # NDCG 및 Hitrate 계산
    for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
        mymodel_pred = user_pred_info.get(str(user), [])
        mymodel_pred = mymodel_pred[:top]
        testset = list(set(np.array(data_info).astype(int)))

        # NDCG 값 계산
        user_ndcg = get_NDCG(mymodel_pred, testset)
        mymodel_ndcg_result[user] = user_ndcg

        # Hitrate 값 계산
        user_hitrate = get_hit_rate(mymodel_pred, testset)
        mymodel_hitrate_result[user] = user_hitrate

    # 평균 NDCG 및 Hitrate 계산
    avg_ndcg = round(np.mean(list(mymodel_ndcg_result.values())), 5)
    avg_hitrate = round(np.mean(list(mymodel_hitrate_result.values())), 5)

    # 결과 출력 및 저장
    print(f"Results -> NDCG: {avg_ndcg}, Hitrate: {avg_hitrate}")

    experiment_results.append({
        'params': params,
        'ndcg': avg_ndcg,
        'hitrate': avg_hitrate
    })

# 최종 결과 출력
print("\nFinal Results:")
for result in experiment_results:
    print(result)

# 최적의 하이퍼파라미터 조합 출력
best_result = max(experiment_results, key=lambda x: (x['ndcg'], x['hitrate']))
print("\nBest Parameters:")
print(best_result)



Running with params: epochs=5, lr=0.0001, dropout=0.3, batch_size=1024, embed_dim=16
Epoch 1/5
704/704 [==============================] - 7s 8ms/step - loss: 0.6451 - binary_crossentropy: 0.6451 - val_loss: 0.5982 - val_binary_crossentropy: 0.5982
Epoch 2/5
704/704 [==============================] - 5s 8ms/step - loss: 0.5675 - binary_crossentropy: 0.5675 - val_loss: 0.5504 - val_binary_crossentropy: 0.5504
Epoch 3/5
704/704 [==============================] - 5s 8ms/step - loss: 0.5390 - binary_crossentropy: 0.5390 - val_loss: 0.5449 - val_binary_crossentropy: 0.5449
Epoch 4/5
704/704 [==============================] - 5s 8ms/step - loss: 0.5344 - binary_crossentropy: 0.5344 - val_loss: 0.5427 - val_binary_crossentropy: 0.5427
Epoch 5/5
704/704 [==============================] - 5s 8ms/step - loss: 0.5329 - binary_crossentropy: 0.5329 - val_loss: 0.5422 - val_binary_crossentropy: 0.5422


100%|██████████| 5994/5994 [00:00<00:00, 6876.39it/s]


Results -> NDCG: 0.66277, Hitrate: 0.63283

Running with params: epochs=5, lr=0.0001, dropout=0.3, batch_size=1024, embed_dim=32
Epoch 1/5
704/704 [==============================] - 9s 10ms/step - loss: 0.6243 - binary_crossentropy: 0.6243 - val_loss: 0.5617 - val_binary_crossentropy: 0.5617
Epoch 2/5
704/704 [==============================] - 7s 10ms/step - loss: 0.5434 - binary_crossentropy: 0.5434 - val_loss: 0.5448 - val_binary_crossentropy: 0.5448
Epoch 3/5
704/704 [==============================] - 7s 10ms/step - loss: 0.5355 - binary_crossentropy: 0.5355 - val_loss: 0.5429 - val_binary_crossentropy: 0.5429
Epoch 4/5
704/704 [==============================] - 7s 10ms/step - loss: 0.5335 - binary_crossentropy: 0.5335 - val_loss: 0.5425 - val_binary_crossentropy: 0.5425
Epoch 5/5
704/704 [==============================] - 7s 10ms/step - loss: 0.5323 - binary_crossentropy: 0.5323 - val_loss: 0.5415 - val_binary_crossentropy: 0.5415


100%|██████████| 5994/5994 [00:00<00:00, 7057.56it/s]


Results -> NDCG: 0.66258, Hitrate: 0.6328

Running with params: epochs=5, lr=0.0001, dropout=0.3, batch_size=2048, embed_dim=16
Epoch 1/5
352/352 [==============================] - 6s 13ms/step - loss: 0.6747 - binary_crossentropy: 0.6747 - val_loss: 0.6340 - val_binary_crossentropy: 0.6340
Epoch 2/5
352/352 [==============================] - 4s 12ms/step - loss: 0.6062 - binary_crossentropy: 0.6062 - val_loss: 0.5962 - val_binary_crossentropy: 0.5962
Epoch 3/5
352/352 [==============================] - 4s 12ms/step - loss: 0.5796 - binary_crossentropy: 0.5796 - val_loss: 0.5625 - val_binary_crossentropy: 0.5625
Epoch 4/5
352/352 [==============================] - 4s 12ms/step - loss: 0.5429 - binary_crossentropy: 0.5429 - val_loss: 0.5468 - val_binary_crossentropy: 0.5468
Epoch 5/5
352/352 [==============================] - 4s 12ms/step - loss: 0.5355 - binary_crossentropy: 0.5355 - val_loss: 0.5440 - val_binary_crossentropy: 0.5440


...

Results -> NDCG: 0.67155, Hitrate: 0.63882

Running with params: epochs=10, lr=0.001, dropout=0.4, batch_size=2048, embed_dim=16
Epoch 1/10
352/352 [==============================] - 6s 13ms/step - loss: 0.5908 - binary_crossentropy: 0.5908 - val_loss: 0.5460 - val_binary_crossentropy: 0.5460
Epoch 2/10
352/352 [==============================] - 4s 12ms/step - loss: 0.5378 - binary_crossentropy: 0.5378 - val_loss: 0.5410 - val_binary_crossentropy: 0.5410
Epoch 3/10
352/352 [==============================] - 4s 12ms/step - loss: 0.5304 - binary_crossentropy: 0.5304 - val_loss: 0.5351 - val_binary_crossentropy: 0.5351
Epoch 4/10
352/352 [==============================] - 4s 12ms/step - loss: 0.5240 - binary_crossentropy: 0.5240 - val_loss: 0.5336 - val_binary_crossentropy: 0.5336
Epoch 5/10
352/352 [==============================] - 4s 12ms/step - loss: 0.5199 - binary_crossentropy: 0.5199 - val_loss: 0.5328 - val_binary_crossentropy: 0.5328
Epoch 6/10
352/352 [==========================

100%|██████████| 5994/5994 [00:00<00:00, 6899.87it/s]


Results -> NDCG: 0.669, Hitrate: 0.63668

Running with params: epochs=10, lr=0.001, dropout=0.4, batch_size=2048, embed_dim=32
Epoch 1/10
352/352 [==============================] - 8s 18ms/step - loss: 0.5779 - binary_crossentropy: 0.5779 - val_loss: 0.5459 - val_binary_crossentropy: 0.5459
Epoch 2/10
352/352 [==============================] - 6s 17ms/step - loss: 0.5358 - binary_crossentropy: 0.5358 - val_loss: 0.5376 - val_binary_crossentropy: 0.5376
Epoch 3/10
352/352 [==============================] - 6s 17ms/step - loss: 0.5270 - binary_crossentropy: 0.5270 - val_loss: 0.5339 - val_binary_crossentropy: 0.5339
Epoch 4/10
352/352 [==============================] - 6s 17ms/step - loss: 0.5217 - binary_crossentropy: 0.5217 - val_loss: 0.5320 - val_binary_crossentropy: 0.5320
Epoch 5/10
352/352 [==============================] - 6s 17ms/step - loss: 0.5169 - binary_crossentropy: 0.5169 - val_loss: 0.5312 - val_binary_crossentropy: 0.5312
Epoch 6/10
352/352 [============================

100%|██████████| 5994/5994 [00:00<00:00, 7099.61it/s]

Results -> NDCG: 0.67295, Hitrate: 0.63947

Final Results:
{'params': (5, 0.0001, 0.3, 1024, 16), 'ndcg': 0.66277, 'hitrate': 0.63283}
{'params': (5, 0.0001, 0.3, 1024, 32), 'ndcg': 0.66258, 'hitrate': 0.6328}
{'params': (5, 0.0001, 0.3, 2048, 16), 'ndcg': 0.66316, 'hitrate': 0.63307}
{'params': (5, 0.0001, 0.3, 2048, 32), 'ndcg': 0.66247, 'hitrate': 0.63311}
{'params': (5, 0.0001, 0.4, 1024, 16), 'ndcg': 0.66237, 'hitrate': 0.6329}
{'params': (5, 0.0001, 0.4, 1024, 32), 'ndcg': 0.66213, 'hitrate': 0.63289}
{'params': (5, 0.0001, 0.4, 2048, 16), 'ndcg': 0.6632, 'hitrate': 0.63307}
{'params': (5, 0.0001, 0.4, 2048, 32), 'ndcg': 0.66274, 'hitrate': 0.63323}
{'params': (5, 0.001, 0.3, 1024, 16), 'ndcg': 0.66703, 'hitrate': 0.6348}
{'params': (5, 0.001, 0.3, 1024, 32), 'ndcg': 0.66929, 'hitrate': 0.63638}
{'params': (5, 0.001, 0.3, 2048, 16), 'ndcg': 0.6644, 'hitrate': 0.63392}
{'params': (5, 0.001, 0.3, 2048, 32), 'ndcg': 0.6677, 'hitrate': 0.63566}
{'params': (5, 0.001, 0.4, 1024, 16), '